# 🚀 ONE-CLICK TTS SERVER

**Просто нажмите "Выполнить всё" (Runtime → Run all) и ждите завершения!**

✅ **Всё запустится автоматически:**
1. Установка зависимостей
2. Запуск TTS сервера
3. Создание публичного URL
4. Тестирование

🎯 **Результат:** Вы получите рабочий URL для использования на сайте.

In [ ]:
# ============================================================================
# 📦 ШАГ 1: УСТАНОВКА ЗАВИСИМОСТЕЙ
# ============================================================================
print("=" * 80)
print("📦 ШАГ 1: УСТАНОВКА ЗАВИСИМОСТЕЙ")
print("=" * 80)

print("Устанавливаем edge-tts и Flask...")
!pip install edge-tts flask -q

print("✅ Зависимости установлены")
print("" * 40)

In [ ]:
# ============================================================================
# 🔧 ШАГ 2: ОЧИСТКА СТАРЫХ ПРОЦЕССОВ
# ============================================================================
print("=" * 80)
print("🔧 ШАГ 2: ОЧИСТКА СТАРЫХ ПРОЦЕССОВ")
print("=" * 80)

import os
import time

print("Убиваем старые процессы...")
os.system('pkill -f "python.*flask" 2>/dev/null || true')
os.system('pkill -f cloudflared 2>/dev/null || true')
os.system('fuser -k 9999/tcp 2>/dev/null || true')

time.sleep(2)
print("✅ Старые процессы убиты")
print("" * 40)

In [ ]:
# ============================================================================
# 🚀 ШАГ 3: ЗАПУСК TTS СЕРВЕРА
# ============================================================================
print("=" * 80)
print("🚀 ШАГ 3: ЗАПУСК TTS СЕРВЕРА")
print("=" * 80)

import subprocess
import asyncio
import threading
from flask import Flask, request, Response
import edge_tts

app = Flask(__name__)

@app.after_request
def cors(response):
    response.headers["Access-Control-Allow-Origin"] = "*"
    return response

@app.route("/tts")
def tts():
    text = request.args.get("text", "")
    voice = request.args.get("voice", "us_young")
    
    if not text:
        return "No text", 400
    
    voices = {
        "us_young": "en-US-ChristopherNeural",
        "us_mature": "en-US-EricNeural", 
        "uk_british": "en-GB-RyanNeural",
        "australian": "en-AU-WilliamNeural",
        "indian": "en-IN-PrabhatNeural"
    }
    
    voice_id = voices.get(voice, voices["us_young"])
    
    async def gen():
        communicate = edge_tts.Communicate(text, voice_id)
        chunks = []
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                chunks.append(chunk["data"])
        return b"".join(chunks)
    
    try:
        audio = asyncio.run(gen())
        return Response(audio, mimetype="audio/mpeg")
    except Exception as e:
        return str(e), 500

@app.route("/")
def home():
    return "TTS Server OK"

def run_server():
    app.run(host="0.0.0.0", port=9999, debug=False, threaded=True)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

time.sleep(3)

# Проверяем что сервер запустился
import requests
try:
    response = requests.get("http://localhost:9999/", timeout=2)
    print(f"✅ Сервер запущен: http://localhost:9999")
    print(f"   Статус: {response.status_code}")
except:
    print("❌ Сервер не отвечает")
    
print("" * 40)

In [ ]:
# ============================================================================
# 🌐 ШАГ 4: СОЗДАНИЕ ПУБЛИЧНОГО URL
# ============================================================================
print("=" * 80)
print("🌐 ШАГ 4: СОЗДАНИЕ ПУБЛИЧНОГО URL")
print("=" * 80)

print("Устанавливаем cloudflared...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb 2>/dev/null || apt-get install -f -y

print("Запускаем туннель...")

def run_cloudflared():
    os.system('cloudflared tunnel --url http://localhost:9999 2>&1 | tee /tmp/cloudflared.log')

tunnel_thread = threading.Thread(target=run_cloudflared, daemon=True)
tunnel_thread.start()

print("Ждем 35 секунд для получения URL...")
for i in range(35, 0, -1):
    if i % 5 == 0:
        print(f"   Осталось: {i} сек...")
    time.sleep(1)

# Ищем URL в логах
import re

with open('/tmp/cloudflared.log', 'r') as f:
    log_content = f.read()
    
url_matches = re.findall(r'https://[a-zA-Z0-9.-]*\.trycloudflare\.com', log_content)
if url_matches:
    url = url_matches[-1]
    print(f"\n✅ ПОЛУЧЕН ПУБЛИЧНЫЙ URL:\n")
    print(f"   🔗 {url}")
    print(f"\n   Пример запроса: {url}/tts?text=Hello&voice=us_young")
    
    # Сохраняем URL
    with open('/content/tts_url.txt', 'w') as f:
        f.write(url)
    print(f"\n💾 URL сохранен в: /content/tts_url.txt")
    
else:
    print("\n❌ URL не найден")
    print("Проверьте логи вручную:")
    print("!tail -50 /tmp/cloudflared.log")
    url = ""

print("" * 40)

In [ ]:
# ============================================================================
# 🧪 ШАГ 5: ФИНАЛЬНЫЙ ТЕСТ
# ============================================================================
print("=" * 80)
print("🧪 ШАГ 5: ФИНАЛЬНЫЙ ТЕСТ")
print("=" * 80)

if 'url' in locals() and url:
    print(f"Тестируем URL: {url}")
    
    # Быстрый тест
    try:
        test_response = requests.get(f"{url}/?text=Test&voice=us_young", timeout=20)
        
        if test_response.status_code == 200:
            print(f"✅ Сервер работает! Статус: {test_response.status_code}")
            print(f"   Размер ответа: {len(test_response.content)} байт")
            
            if len(test_response.content) > 1000:
                print(f"   🎵 Получено аудио!")
            
            print("\n" + "=" * 80)
            print("🎉 ВСЁ ГОТОВО!")
            print("=" * 80)
            print(f"\nВаш рабочий URL: {url}")
            print(f"\nИспользуйте этот URL на вашем сайте!")
            print(f"\nПримеры запросов:")
            print(f"1. {url}/tts?text=Hello&voice=us_young")
            print(f"2. {url}/?text=Привет&voice=us_mature")
            print(f"3. {url}/tts?text=Long+text+here&voice=uk_british")
            
        else:
            print(f"❌ Сервер отвечает с ошибкой: {test_response.status_code}")
            
    except Exception as e:
        print(f"❌ Ошибка тестирования: {e}")
        
else:
    print("❌ URL не получен, проверьте предыдущие шаги")

print("\n" + "=" * 80)
print("📋 ИНСТРУКЦИЯ ДЛЯ САЙТА:")
print("=" * 80)
print("\n1. Скопируйте URL выше")
print("2. На вашем сайте вставьте его в поле 'Server URL'")
print("3. Нажмите 'Update' и 'Test Connection'")
print("4. Если всё ок - используйте генерацию!")
print("\n💡 URL будет работать 24-48 часов")
print("   Для продления перезапустите этот ноутбук")
print("=" * 80)